In [19]:
!pip install Dbias


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf

from Dbias.text_debiasing import *;
from Dbias.bias_classification import *;
from Dbias.bias_recognition import *;
from Dbias.bias_masking import *;
import os



from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

ModuleNotFoundError: No module named 'Dbias'

In [11]:
def custom_classification(x):
    classi_out = classify(x)
    return classi_out[0]['label'], classi_out[0]['score']

def custom_recognizer(x):
    biased_words = recognizer(x)
    biased_words_list = []
    for id in range(0, len(biased_words)):
        biased_words_list.append(biased_words[id]['entity'])
    return ", ".join(biased_words_list)

def custom_debiasing(x):
    suggestions = run(x)
    if suggestions == None:
        return ""
    else:
      all_suggestions = []
      for sent in suggestions[0:3]:
        all_suggestions.append(sent['Sentence'])
      return "\n\n".join(all_suggestions)

In [3]:
# Load gender bias headlines

headlines = pd.read_csv("gender_bias_data.csv")

print(headlines.columns)

Index(['headline_no_site'], dtype='object')


In [14]:
headlines['prediction'] = headlines['headline_no_site'].apply(classifier)

# Extract the label and score from the prediction
headlines['state'] = headlines['prediction'].apply(lambda x: x[0]['label'])
headlines['score'] = headlines['prediction'].apply(lambda x: x[0]['score'])

# Drop the 'prediction' column if you don't need it anymore
headlines = headlines.drop(columns=['prediction'])

# Print the resulting DataFrame with bias predictions
print(headlines[['headline_no_site', 'state', 'score']])


headlines[['state', 'probability']] = headlines['headline_no_site'].apply(lambda x: custom_classification(x)).to_list()
headlines

                                     headline_no_site       state     score
0   Life sentence for Plymouth woman who murdered ...  Non-biased  0.980371
1   “I Had Never Even Heard Of Her Before She Cure...  Non-biased  0.681668
2   'Honey Boo Boo' Mom Wants Female Pres as Hilla...      Biased  0.892100
3   New York Woman Says Rihanna's Lipstick Gave He...      Biased  0.781223
4   Teens Notice Pretty Blonde Woman In Car Beside...      Biased  0.992805
5   Crime Scene   Mother charged after Pr. William...  Non-biased  0.873353
6   She Had The Meanest Mom In The World. Now? She...      Biased  0.990178
7   A US woman on why she disagrees with Obama on ...      Biased  0.900038
8   EastEnders star Emer Kenny’s sister told she w...      Biased  0.533608
9   Army Colonel: Use 'Average Looking Women' to P...      Biased  0.639923
10  Size 30 woman ditches cruel husband who called...      Biased  0.969968
11  Mom Hopes For A Normal Baby, But When Doctors ...      Biased  0.995048
12  Watch th

NameError: name 'classify' is not defined

In [10]:
classifier("Nevertheless, Trump and other Republicans have tarred the protests as havens for terrorists intent on destroying property.")

[{'label': 'Biased', 'score': 0.9938021898269653}]